In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import os
import time
import random

In [2]:
def read_top_instance(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        n_line = file.readline().strip()
        m_line = file.readline().strip()
        tmax_line = file.readline().strip()

        # Parse n N
        _, N = n_line.split()
        N = int(N)

        # Parse m P
        _, P = m_line.split()
        P = int(P)

        # Parse tmax Tmax
        _, Tmax = tmax_line.split()
        Tmax = float(Tmax)

        # Read the remaining lines for each node
        nodes = []
        for i in range(N):
            line = file.readline().strip()
            if line == '':
                continue  # Skip empty lines
            x_str, y_str, s_str = line.split()
            x = float(x_str)
            y = float(y_str)
            s = float(s_str)
            nodes.append({'id': i, 'x': x, 'y': y, 'score': s})
        
        return N, P, Tmax, nodes


In [3]:

def compute_distance_matrix(nodes):
    N = len(nodes)
    distance_matrix = np.zeros((N, N))
    for i in range(N):
        xi, yi = nodes[i]['x'], nodes[i]['y']
        for j in range(N):
            xj, yj = nodes[j]['x'], nodes[j]['y']
            distance = np.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix


In [4]:
import numpy as np
import random

def generate_dynamic_travel_times(nodes, base_travel_time, time_window, seed=None):
    """
    Generates dynamic travel time parameters for each arc based on randomly selected distributions.
    
    Enhancements:
    - Limits the magnitude of slopes to prevent excessive changes in travel times.
    - Ensures travel times remain within [0.8 * base_time, 1.2 * base_time].
    
    :param nodes: List of node identifiers.
    :param base_travel_time: Dictionary of dictionaries with base travel times, base_travel_time[i][j] = static time from i to j.
    :param time_window: Tuple indicating the time window (start_time, end_time).
    :param seed: Random seed for reproducibility.
    :return: Dictionary with keys as (i, j) and values as dictionaries containing distribution type and parameters.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    distribution_types = ['decreasing', 'increasing', 'v_shaped', 'inverse_v_shaped']
    dynamic_travel_time = {}
    
    start_time, end_time = time_window
    median_time = (start_time + end_time) / 2
    total_time = end_time - start_time
    
    # Define slope limits
    slope_min = -0.05
    slope_max = 0.05
    
    node_id = [node['id'] for node in nodes]
    
    for i in node_id:
        for j in node_id:
            if i == j:
                continue  # No travel time needed for the same node
            distribution = random.choice(distribution_types)
            base_time = base_travel_time[i][j]
            # Calculate travel time bounds
            min_travel_time = 0.9 * base_time
            max_travel_time = 1.1 * base_time
            
            if distribution == 'decreasing':
                # Travel time decreases linearly from max_travel_time at start_time to base_time at end_time
                a_ij = (base_time - max_travel_time) / total_time  # Negative slope
                # Clamp slope within [slope_min, slope_max]
                a_ij = max(slope_min, min(a_ij, slope_max))
                b_ij = max_travel_time - a_ij * start_time
                dynamic_travel_time[(i, j)] = {
                    'distribution': 'decreasing',
                    'a': a_ij,
                    'b': b_ij
                }
            elif distribution == 'increasing':
                # Travel time increases linearly from base_time at start_time to max_travel_time at end_time
                a_ij = (max_travel_time - base_time) / total_time  # Positive slope
                # Clamp slope within [slope_min, slope_max]
                a_ij = max(slope_min, min(a_ij, slope_max))
                b_ij = base_time - a_ij * start_time
                dynamic_travel_time[(i, j)] = {
                    'distribution': 'increasing',
                    'a': a_ij,
                    'b': b_ij
                }
            elif distribution == 'v_shaped':
                # Travel time decreases to min_travel_time at midpoint, then increases to max_travel_time
                # Define two segments
                # Randomly select a midpoint around the median within ±10% of total_time
                midpoint = median_time + random.uniform(-0.1 * total_time, 0.1 * total_time)
                
                # Segment 1: start_time to midpoint
                a1 = (min_travel_time - max_travel_time) / (midpoint - start_time)  # Negative slope
                a1 = max(slope_min, min(a1, slope_max))
                b1 = max_travel_time - a1 * start_time
                
                # Segment 2: midpoint to end_time
                a2 = (max_travel_time - min_travel_time) / (end_time - midpoint)  # Positive slope
                a2 = max(slope_min, min(a2, slope_max))
                b2 = min_travel_time - a2 * midpoint
                
                dynamic_travel_time[(i, j)] = {
                    'distribution': 'v_shaped',
                    'segments': [
                        {'slope': a1, 'intercept': b1, 'start': start_time, 'end': midpoint},
                        {'slope': a2, 'intercept': b2, 'start': midpoint, 'end': end_time}
                    ],
                    'midpoint': midpoint
                }
            elif distribution == 'inverse_v_shaped':
                # Travel time increases to max_travel_time at midpoint, then decreases to min_travel_time
                # Define two segments
                # Randomly select a midpoint around the median within ±10% of total_time
                midpoint = median_time + random.uniform(-0.1 * total_time, 0.1 * total_time)
                
                # Segment 1: start_time to midpoint
                a1 = (max_travel_time - base_time) / (midpoint - start_time)  # Positive slope
                a1 = max(slope_min, min(a1, slope_max))
                b1 = base_time - a1 * start_time
                
                # Segment 2: midpoint to end_time
                a2 = (min_travel_time - max_travel_time) / (end_time - midpoint)  # Negative slope
                a2 = max(slope_min, min(a2, slope_max))
                b2 = max_travel_time - a2 * end_time
                
                dynamic_travel_time[(i, j)] = {
                    'distribution': 'inverse_v_shaped',
                    'segments': [
                        {'slope': a1, 'intercept': b1, 'start': start_time, 'end': midpoint},
                        {'slope': a2, 'intercept': b2, 'start': midpoint, 'end': end_time}
                    ],
                    'midpoint': midpoint
                }
            else:
                # Default to base time if distribution type is unknown
                dynamic_travel_time[(i, j)] = {
                    'distribution': 'constant',
                    'a': 0,
                    'b': base_time
                }
    
    return dynamic_travel_time


In [9]:
def solve_top_dtt_gurobi(N, P, Tmax, nodes, distance_matrix, seed=None):
    """
    Solves the Team Orienteering Problem (TOP) using Gurobi with dynamic travel times.

    :param N: Number of nodes.
    :param P: Number of vehicles.
    :param Tmax: Maximum allowable time for any route.
    :param nodes: List of node identifiers.
    :param distance_matrix: Dictionary of dictionaries with base travel times.
    :param seed: Random seed for reproducibility.
    :return: Tuple containing routes, run_time, optimality_gap, total_collected_prize.
    """
    try:
        # Start timing
        start_time = time.time()
    
        # Generate dynamic travel times
        time_window = (0, Tmax)  # Assuming operations start at time 0 and end at Tmax
        dynamic_travel_time = generate_dynamic_travel_times(nodes, distance_matrix, time_window, seed)
    
        # Create a new model
        model = gp.Model('TOP_dynamic_travel_time')
        model.Params.TimeLimit = 600  # Set a time limit of 10 minutes
        model.Params.MIPGap = 0.01    # Set a MIP gap tolerance of 1%
        model.Params.OutputFlag = 0    # Disable Gurobi output
    
        # Sets
        V = range(N)  # Set of nodes
        K = range(P)  # Set of vehicles
    
        # Parameters
        p = [node['score'] for node in nodes]
    
        # Decision variables
        x = model.addVars(V, V, K, vtype=GRB.BINARY, name='x')  # Arc variables
        y = model.addVars(V, K, vtype=GRB.BINARY, name='y')    # Node visit variables
        # Auxiliary variables for MTZ constraints
        u = model.addVars(V, K, vtype=GRB.CONTINUOUS, lb=0, ub=Tmax, name='u')
    
        # Binary variables for piecewise segments
        z = {}
        for i in V:
            for j in V:
                if i == j:
                    continue
                dist_info = dynamic_travel_time.get((i, j), {'distribution': 'constant', 'a': 0, 'b': distance_matrix[i][j]})
                if dist_info['distribution'] in ['v_shaped', 'inverse_v_shaped']:
                    z[(i, j)] = model.addVars(2, vtype=GRB.BINARY, name=f"z_{i}_{j}_s")
    
        # Objective function: Maximize total collected scores
        model.setObjective(
            gp.quicksum(p[j] * y[j, k] for j in V for k in K),
            GRB.MAXIMIZE
        )
    
        # Constraints
        for k in K:
            # Flow conservation constraints
            for j in V:
                if j != 0 and j != N - 1:
                    # Incoming arcs equal to y[j, k]
                    model.addConstr(
                        gp.quicksum(x[i, j, k] for i in V if i != j) == y[j, k],
                        f"FlowIn_{j}_{k}"
                    )
                    # Outgoing arcs equal to y[j, k]
                    model.addConstr(
                        gp.quicksum(x[j, i, k] for i in V if i != j) == y[j, k],
                        f"FlowOut_{j}_{k}"
                    )
    
            # Start at depot (node 0)
            model.addConstr(
                gp.quicksum(x[0, j, k] for j in V if j != 0) == 1,
                f"StartAtDepot_{k}"
            )
    
            # End at depot (node N-1)
            model.addConstr(
                gp.quicksum(x[j, N - 1, k] for j in V if j != N - 1) == 1,
                f"EndAtDepot_{k}"
            )
    
            # No flow into depot
            model.addConstr(
                gp.quicksum(x[j, 0, k] for j in V if j != 0) == 0,
                f"NoFlowIntoDepot_{k}"
            )
    
            # No flow out of depot
            model.addConstr(
                gp.quicksum(x[N - 1, j, k] for j in V if j != N - 1) == 0,
                f"NoFlowOutOfDepot_{k}"
            )
    
            # Time budget constraints
            # To handle dynamic travel times, we need to model arrival times
            # This is done through MTZ constraints with dynamic travel times
    
            # Flow conservation for time
            for i in V:
                if i == 0:
                    model.addConstr(
                        u[i, k] == 0,
                        f"StartTime_{i}_{k}"
                    )
                    continue
                model.addConstr(
                    u[i, k] >= 0,
                    f"ArrivalTimeLowerBound_{i}_{k}"
                )
                model.addConstr(
                    u[i, k] <= Tmax,
                    f"ArrivalTimeUpperBound_{i}_{k}"
                )
    
            # Time and MTZ constraints
            for i in V:
                for j in V:
                    if i == j:
                        continue
                    dist_info = dynamic_travel_time.get((i, j), {'distribution': 'constant', 'a': 0, 'b': distance_matrix[i][j]})
                    if dist_info['distribution'] == 'constant':
                        a_ij = 0
                        b_ij = dist_info['b']
                        model.addConstr(
                            u[j, k] >= u[i, k] + a_ij * u[i, k] + b_ij  - 10*Tmax * (1 - x[i, j, k]),
                            f"Time_{i}_{j}_{k}"
                        )
                    elif dist_info['distribution'] in ['decreasing', 'increasing']:
                        a_ij = dist_info['a']
                        b_ij = dist_info['b']
                        model.addConstr(
                            u[j, k] >= u[i, k] + a_ij * u[i, k] + b_ij - 10*Tmax * (1 - x[i, j, k]),
                            f"Time_{i}_{j}_{k}"
                        )
                    elif dist_info['distribution'] in ['v_shaped', 'inverse_v_shaped']:
                        # Piecewise linear constraints with binary variables z[(i,j)][0] and z[(i,j)][1]
                        # Ensure that only one segment is active
                        model.addConstr(
                            z[(i, j)][0] + z[(i, j)][1] == x[i, j, k],
                            f"Piecewise_Activation_{i}_{j}_{k}"
                        )
    
                        # If first segment is active
                        segment1 = dist_info['segments'][0]
                        slope1 = segment1['slope']
                        intercept1 = segment1['intercept']
                        start1 = segment1['start']
                        end1 = segment1['end']
    
                        model.addConstr(
                            u[j, k] >= u[i, k] + slope1 * u[i, k] + intercept1  - 10*Tmax * (1 - z[(i, j)][0]),
                            f"Piecewise_Segment1_{i}_{j}_{k}"
                        )
    
                        # If second segment is active
                        segment2 = dist_info['segments'][1]
                        slope2 = segment2['slope']
                        intercept2 = segment2['intercept']
                        start2 = segment2['start']
                        end2 = segment2['end']
    
                        model.addConstr(
                            u[j, k] >= u[i, k] + slope2 * u[i, k] + intercept2 +  - 10*Tmax * (1 - z[(i, j)][1]),
                            f"Piecewise_Segment2_{i}_{j}_{k}"
                        )
    
        # Each node is visited at most once across all vehicles
        for j in V:
            if j != 0 and j != N - 1:
                model.addConstr(
                    gp.quicksum(y[j, k] for k in K) <= 1,
                    f"VisitOnce_{j}"
                )

        # Optimize the model
        model.optimize()
        end_time = time.time()
        run_time = end_time - start_time

        # Initialize the routes dictionary and collected prize
        routes = {}
        total_collected_prize = 0

        # Extract the solution
        if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
            binding = [c for c in model.getConstrs() if abs(c.Slack) < 1e-6]
            #print(binding)
            for k in K:
                # Get the arcs used by vehicle k
                arcs = [(i, j) for i in V for j in V if i != j and x[i, j, k].X > 0.5]
                if not arcs:
                    continue
                # Store the arcs in the routes dictionary
                routes[k] = arcs

                # Reconstruct the route to calculate the collected prize
                arc_dict = {i: j for (i, j) in arcs}
                route = [0]
                current_node = 0
                while current_node != N - 1:
                    if current_node in arc_dict:
                        next_node = arc_dict[current_node]
                        route.append(next_node)
                        current_node = next_node
                    else:
                        break
                # Calculate collected prize for this vehicle
                collected_prize = sum(p[j] for j in route if j != 0 and j != N - 1)
                total_collected_prize += collected_prize
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            return None, None, None, None
        else:
            print(f"Optimization was stopped with status {model.Status}")
            return None, None, None, None

        # Get the optimality gap
        if model.status == GRB.OPTIMAL:
            gap = 0.0
        else:
            gap = model.MIPGap

        return routes, run_time, gap, total_collected_prize
    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None, None, None, None
    except AttributeError as e:
        print(f"Attribute error: {e}")
        return None, None, None, None


In [7]:
def process_all_instances(directory_path):
    import csv
    import os

    # Prepare a list to store the results
    results = []

    csv_file = 'results.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Instance', 'RunTime', 'Gap', 'TotalCollectedPrize', 'AveragePrizePerVehicle'])

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Adjust the extension if needed
            file_path = os.path.join(directory_path, filename)
            print(f"\nProcessing instance: {filename}")
            N, P, Tmax, nodes = read_top_instance(file_path)
            distance_matrix = compute_distance_matrix(nodes)
            routes, run_time, gap, total_collected_prize = solve_top_dtt_gurobi(
                                                                                N=N,
                                                                                P=P,
                                                                                Tmax=Tmax,
                                                                                nodes=nodes,
                                                                                distance_matrix=distance_matrix,
                                                                                seed=42
                                                                                )
                                    
            if routes is not None:
                # Calculate average prize per vehicle
                num_vehicles_used = len(routes)
                if num_vehicles_used > 0:
                    average_prize = total_collected_prize / num_vehicles_used
                else:
                    average_prize = 0

                # Append the results
                results.append({
                    'Instance': filename,
                    'RunTime': run_time,
                    'Gap': gap,
                    'TotalCollectedPrize': total_collected_prize,
                    'AveragePrizePerVehicle': average_prize
                })

                # Write the results to the CSV file
                with open(csv_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([filename, run_time, gap, total_collected_prize, average_prize])
            else:
                print(f"No solution found for instance {filename}.")

    return results

In [8]:
#test with first instance
if __name__ == "__main__":
    # Path to a single instance file
    instance_file = 'Set_100_234/p4.2.a.txt'  

    # Read the instance data
    N, P, Tmax, nodes = read_top_instance(instance_file)

    # Compute the distance matrix
    distance_matrix = compute_distance_matrix(nodes)

    # Solve the TOP using Gurobi
    results  = solve_top_dtt_gurobi(N, P, Tmax, nodes, distance_matrix)
    print(results)

Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40624 rows, 30364 columns and 159770 nonzeros
Model fingerprint: 0x7d4a3e42
Variable types: 200 continuous, 30164 integer (30164 binary)
Coefficient statistics:
  Matrix range     [9e-01, 3e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 33292 rows and 23132 columns
Presolve time: 1.54s
Presolved: 7332 rows, 7232 columns, 34626 nonzeros
Variable types: 196 continuous, 7036 integer (7036 binary)

Root relaxation: objective 1.306000e+03, 1168 iterations, 0.13 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objectiv

In [10]:

if __name__ == "__main__":
    # Set the path to your instance directory
    instance_directory = 'Set_100_234'  

    # Process all instances in the directory and collect results
    results = process_all_instances(instance_directory)

    print("\nSummary of Results:")
    for result in results:
        print(f"Instance: {result['Instance']}, Run Time: {result['RunTime']:.2f}s, "
              f"Gap: {result['Gap']*100:.2f}%, Total Prize: {result['TotalCollectedPrize']}, "
              f"Avg Prize/Vehicle: {result['AveragePrizePerVehicle']}")


Processing instance: p4.2.a.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40608 rows, 30356 columns and 159722 nonzeros
Model fingerprint: 0x9f570f41
Variable types: 200 continuous, 30156 integer (30156 binary)
Coefficient statistics:
  Matrix range     [9e-01, 3e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 33274 rows and 23123 columns
Presolve time: 1.55s
Presolved: 7334 rows, 7233 columns, 34630 nonzeros
Variable types: 196 continuous, 7037 integer (7037 binary)

Root relaxation: objective 1.306000e+03, 1242 iterations, 0.14 seconds (0.11 work units)

    Nodes    | 